In [1]:
import string
import numpy as np
from PIL import Image
import os
from pickle import dump, load
import numpy as np
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras_hub.tokenizers import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout
# small library for seeing the progress of loops.
from tqdm import tqdm_notebook as tqdm
#tqdm().pandas()

In [2]:
# --- USER-OVERRIDE: Corrected hard-coded paths (token filename includes leading dot+underscore) ---
# Set this to the exact folder where your Flickr8k text files live (from your screenshots)
dataset_text = '/content/drive/My Drive/Flickr8k_text'   # <-- corrected to the folder shown in your screenshots

# Exact filenames (edit if necessary)
train_list_file = dataset_text + '/Flickr_8k.trainImages.txt'
dev_list_file   = dataset_text + '/Flickr_8k.devImages.txt'
test_list_file  = dataset_text + '/Flickr_8k.testImages.txt'
descriptions_file = dataset_text + '/ExpertAnnotations.txt'

# NOTE: your Drive shows a file named '._Flickr8k.lemma.token.txt' (leading dot+underscore).
# Use that exact name so the notebook won't fail to find it.
token_file = dataset_text + '/._Flickr8k.lemma.token.txt'   # corrected exact filename

# features file (change if located elsewhere)
features_file = dataset_text + '/features.p'

print("\n--- Using hard-coded paths now ---")
print(" dataset_text      =", dataset_text)
print(" descriptions_file =", descriptions_file)
print(" token_file        =", token_file)
print(" features_file     =", features_file)

# Print directory contents so you can visually confirm the filenames in Colab
print("\nFiles inside dataset_text folder:")
import os
try:
    for f in sorted(os.listdir(dataset_text)):
        print(' -', f)
except Exception as e:
    print('Could not list folder contents:', e)
    print('Double-check that you mounted Drive in Colab and that the folder path above is exact.')



--- Using hard-coded paths now ---
 dataset_text      = /content/drive/My Drive/Flickr8k_text
 descriptions_file = /content/drive/My Drive/Flickr8k_text/ExpertAnnotations.txt
 token_file        = /content/drive/My Drive/Flickr8k_text/._Flickr8k.lemma.token.txt
 features_file     = /content/drive/My Drive/Flickr8k_text/features.p

Files inside dataset_text folder:
Could not list folder contents: [Errno 2] No such file or directory: '/content/drive/My Drive/Flickr8k_text'
Double-check that you mounted Drive in Colab and that the folder path above is exact.


In [3]:
# --- Dataset setup (Google Drive + Flickr8k paths) ---
from google.colab import drive
drive.mount('/content/drive')

# Set this to the folder in your Google Drive that contains the Flickr files
dataset_text = '/content/drive/My Drive/Flickr8k_text'  # <-- change if your folder name differs

# Use the exact filenames from your Drive
train_list_file = dataset_text + '/Flickr_8k.trainImages.txt'
dev_list_file = dataset_text + '/Flickr_8k.devImages.txt'
test_list_file = dataset_text + '/Flickr_8k.testImages.txt'

# Captions file (annotations)
descriptions_file = dataset_text + '/ExpertAnnotations.txt'

# Token file (lemma tokens)
token_file = dataset_text + '/Flickr8k.lemma.token.txt'

# Features file (update this path if features.p is in another folder)
features_file = dataset_text + '/features.p'


Mounted at /content/drive


In [4]:
# Loading a text file into memory
def load_doc(filename):
    # Opening the file as read only
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text
# get all imgs with their captions
def all_img_captions(filename):
    file = load_doc(filename)
    captions = file.split('\n')
    descriptions ={}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [ caption ]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions
#Data cleaning- lower casing, removing puntuations and words containing numbers
def cleaning_text(captions):
    table = str.maketrans('','',string.punctuation)
    for img,caps in captions.items():
        for i,img_caption in enumerate(caps):
            img_caption = img_caption.replace("-"," ")
            desc = img_caption.split()
            #converts to lowercase
            desc = [word.lower() for word in desc]
            #remove punctuation from each token
            desc = [word.translate(table) for word in desc]
            #remove hanging 's and a
            desc = [word for word in desc if(len(word)>1)]
            #remove tokens with numbers in them
            desc = [word for word in desc if(word.isalpha())]
            #convert back to string
            img_caption = ' '.join(desc)
            captions[img][i]= img_caption
    return captions
def text_vocabulary(descriptions):
    # build vocabulary of all unique words
    vocab = set()
    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]
    return vocab
#All descriptions in one file
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc )
    data = "\n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()
# Set these path according to project folder in you system
# dataset_text = "/content/drive/My Drive/Colab Notebooks/flicker8k" # This line is not needed as dataset_text is defined in a previous cell
dataset_images = "/content/drive/MyDrive/flicker8k/Images"
#we prepare our text data
filename = token_file # Use the token_file variable from the previous cell
#loading the file that contains all data
#mapping them into descriptions dictionary img to 5 captions
descriptions = all_img_captions(filename)
print("Length of descriptions =" ,len(descriptions))
#cleaning the descriptions
clean_descriptions = cleaning_text(descriptions)
#building vocabulary
vocabulary = text_vocabulary(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))
#saving each description to file
save_descriptions(clean_descriptions, dataset_text + "/" +"descriptions.txt")

Length of descriptions = 8092
Length of vocabulary =  6375


In [5]:
import pickle

with open("/content/drive/MyDrive/flicker8k/features.p", "rb") as f:
    features = pickle.load(f)

print("✅ Features loaded:", len(features))


✅ Features loaded: 8091


In [7]:
#load the data
def load_photos(filename):
    file = load_doc(filename)
    photos = file.split("\n")[:-1]
    return photos
def load_clean_descriptions(filename, photos):
    #loading clean_descriptions
    file = load_doc(filename)
    descriptions = {}
    for line in file.split("\n"):
        words = line.split()
        if len(words)<1 :
            continue
        image, image_caption = words[0], words[1:]
        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = '<start> ' + " ".join(image_caption) + ' <end>'
            descriptions[image].append(desc)
    return descriptions
def load_features(photos):
    #loading all features
    all_features = load(open("/content/drive/MyDrive/flicker8k/features.p","rb"))
    #selecting only needed features
    features = {k:all_features[k] for k in photos}
    return features
filename = dataset_text + "/" + "Flickr_8k.trainImages.txt"
#train = loading_data(filename)
train_imgs = load_photos(filename)
train_descriptions = load_clean_descriptions(dataset_text + "/" +"descriptions.txt", train_imgs)
train_features = load_features(train_imgs)

In [8]:
#converting dictionary to clean list of descriptions
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc
#creating tokenizer class
#this will vectorise text corpus
#each integer will represent token in dictionary
#from keras_hub.tokenizers import Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer
def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer
# give each word an index, and store that into tokenizer.p pickle file
tokenizer = create_tokenizer(train_descriptions)

# Construct the full path for the tokenizer file using dataset_text
tokenizer_path = os.path.join(dataset_text, 'tokenizer.p')

# Save the tokenizer to the constructed path
dump(tokenizer, open(tokenizer_path, 'wb'))

vocab_size = len(tokenizer.word_index) + 1
vocab_size

5520

In [9]:
#calculate maximum length of descriptions
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)

max_length = max_length(descriptions)
max_length

33

In [10]:
def data_generator(descriptions, features, tokenizer, max_length):
    while 1:
        for key, description_list in descriptions.items():
            #retrieve photo features
            feature = features[key] # Get the feature array
            # Ensure the feature is a 1D array of size 2048
            if feature.shape == (1, 2048):
                feature = np.squeeze(feature)
            elif feature.shape != (2048,):
                 # Reshape to (2048,) if it's not already
                 feature = feature.reshape(2048,)


            # walk through each description for the image
            for desc in description_list:
                # encode the sequence
                seq = tokenizer.texts_to_sequences([desc])[0]
                # split one sequence into multiple X,y pairs
                for i in range(1, len(seq)):
                    # split into input and output pair
                    in_seq, out_seq = seq[:i], seq[i]
                    # pad input sequence
                    in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
                    # encode output sequence
                    out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
                    # yield individual sample with expanded feature dimension
                    yield (np.expand_dims(feature, axis=0), np.expand_dims(in_seq, axis=0)), np.expand_dims(out_seq, axis=0)

In [11]:
from keras.utils import plot_model
# define the captioning model
def define_model(vocab_size, max_length):
    # features from the CNN model squeezed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    fe1 = Dropout(0.5)(inputs1)
    fe2 = Dense(256, activation='relu')(fe1)
    # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    se2 = Dropout(0.5)(se1)
    se3 = LSTM(256, use_cudnn=False)(se2) # Add use_cudnn=False here
    # Merging both models
    decoder1 = add([fe2, se3])
    decoder2 = Dense(256, activation='relu')(decoder1)
    outputs = Dense(vocab_size, activation='softmax')(decoder2)
    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    # summarize model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)
    return model

In [13]:
# train our model
print('Dataset: ', len(train_imgs))
print('Descriptions: train=', len(train_descriptions))
print('Photos: train=', len(train_features))
print('Vocabulary Size:', vocab_size)
print('Description Length: ', max_length)
model = define_model(vocab_size, max_length)
epochs = 50
steps = len(train_descriptions)
# making a directory models to save our models
os.makedirs("models",exist_ok=True)
for i in range(epochs):
    generator = data_generator(train_descriptions, train_features, tokenizer, max_length)
    model.fit(generator, epochs=1, steps_per_epoch= steps, verbose=1)
    model.save("models/model_" + str(i) + ".h5")

Dataset:  6000
Descriptions: train= 6000
Photos: train= 6000
Vocabulary Size: 5520
Description Length:  33


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 33)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_2       │ (None, 2048)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 33, 256)   │  1,413,120 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 2048)      │          0 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 33, 256)   │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal_1         │ (None, 33)        │          0 │ input_layer_3[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 256)       │    524,544 │ dropout_2[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_1 (LSTM)       │ (None, 256)       │    525,312 │ dropout_3[0][0],  │
│                     │                   │            │ not_equal_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 256)       │          0 │ dense_3[0][0],    │
│                     │                   │            │ lstm_1[0][0]      │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │     65,792 │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 5520)      │  1,418,640 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 3,947,408 (15.06 MB)

 Trainable params: 3,947,408 (15.06 MB)

...

6000/6000 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - loss: 1.2606


6000/6000 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - loss: 1.2336


6000/6000 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - loss: 1.2067


6000/6000 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - loss: 1.2211


6000/6000 ━━━━━━━━━━━━━━━━━━━━ 29s 5ms/step - loss: 1.2544


In [16]:
# Save both weights and full model
model.save_weights("/content/drive/MyDrive/flickr8k/caption_weights_50.weights.h5")
model.save("/content/drive/MyDrive/flickr8k/caption_model.keras")
print("✅ Model saved: weights (.h5) + full TF model")

✅ Model saved: weights (.h5) + full TF model
